In [2]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
import requests
from functions import *

2023-03-15 12:25:05.303002: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 12:25:05.445825: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-15 12:25:06.326357: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/bwhpc/common/devel/cuda/11.8/lib64
2023-03-15 12:25:06.326439: W tensorflow/compil

In [2]:
use_cuda = torch.cuda.is_available()
use_cuda

True

In [4]:
# Load the BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
model = model.to("cuda:0")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
df_entity2text = pd.read_csv('data/wn18rr_entity2text.txt', delimiter="\t", header = None, names=["id", "definition"])
df_entity2text[["entity", "description"]] = df_entity2text["definition"].str.split(',', n=1, expand=True)
df_entity2text[:5]

,id,definition,entity,description
0,14854262,"stool, solid excretory product evacuated from ...",stool,solid excretory product evacuated from the bo...
1,590383,"chieftainship, the position of chieftain",chieftainship,the position of chieftain
2,8769179,"saxony, an area in Germany around the upper El...",saxony,an area in Germany around the upper Elbe rive...
3,2338145,"ondatra zibethica, beaver-like aquatic rodent ...",ondatra zibethica,beaver-like aquatic rodent of North America w...
4,1990168,"founder, sink below the surface",founder,sink below the surface


In [8]:
# Tokenize the KG entities
input_to_tokens = tokenizer(df_entity2text["entity"].to_list(), padding=True, return_tensors='pt')['input_ids']
input_to_tokens.device

device(type='cpu')

In [9]:
cd = customdata(input_to_tokens)
dataloader = torch.utils.data.DataLoader(cd, batch_size = 2500, shuffle = False, num_workers = 0)

#### Generate BERT embeddings in batches 

In [16]:
iter = 1
embeddings = []
for data in dataloader:
    data = data.to("cuda:0")
    output = generate_bert_embeddings(data,model)
    output = torch.stack(output, 0)# output is originally tuple → stack BERT hidden layers (13) tensors into 1 tensor    

    if iter < 10:
        batch_nr = "0" + str(iter)
    else:
        batch_nr = str(iter)
    #torch.save(output, "nlm_embeddings/bert_wn18rr/" + batch_nr + "_bert_4lastlayers_wn18rr_ent.pt")
    #embeddings.append(output)
    
    
    
    #Additional Info when using cuda
    print('Memory Usage in Iteration:', iter,
          '| Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB |', 
          'Cached: ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')
    
    iter = iter + 1
    del data
    del output
    torch.cuda.empty_cache()

Memory Usage in Iteration: 1 | Allocated: 27.8 GB | Cached:  28.2 GB
Memory Usage in Iteration: 2 | Allocated: 27.8 GB | Cached:  28.2 GB
Memory Usage in Iteration: 3 | Allocated: 27.8 GB | Cached:  28.2 GB
Memory Usage in Iteration: 4 | Allocated: 27.8 GB | Cached:  28.2 GB
Memory Usage in Iteration: 5 | Allocated: 27.8 GB | Cached:  28.2 GB
Memory Usage in Iteration: 6 | Allocated: 27.8 GB | Cached:  28.2 GB
Memory Usage in Iteration: 7 | Allocated: 27.8 GB | Cached:  28.2 GB
Memory Usage in Iteration: 8 | Allocated: 27.8 GB | Cached:  28.2 GB
Memory Usage in Iteration: 9 | Allocated: 27.8 GB | Cached:  28.2 GB
Memory Usage in Iteration: 10 | Allocated: 27.8 GB | Cached:  28.2 GB
Memory Usage in Iteration: 11 | Allocated: 27.8 GB | Cached:  28.2 GB
Memory Usage in Iteration: 12 | Allocated: 27.8 GB | Cached:  28.2 GB
Memory Usage in Iteration: 13 | Allocated: 27.8 GB | Cached:  28.2 GB
Memory Usage in Iteration: 14 | Allocated: 27.8 GB | Cached:  28.2 GB
Memory Usage in Iteration: 15

In [13]:
torch.cuda.empty_cache()

In [14]:
print('| Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB |', 
          'Cached: ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

| Allocated: 0.4 GB | Cached:  0.5 GB


In [11]:
#torch.save(embeddings,'/nlm_embeddings/bert_wn18rr/bert_4lastlayers_wn18rr_ent.pt')